In [ ]:
!pip3 install pyro-ppl==0.2.1

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
from IPython import display
import os
from PIL import Image
from torch.utils.data.dataset import Dataset

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable



%matplotlib inline

In [ ]:
# Check Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

Load the data

In [ ]:
batch_size = 500

AlexTransform = transforms.Compose([
    transforms.Resize((227, 227)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('mnist-data/', train=True, download=True, transform=AlexTransform),
        batch_size=500, shuffle=True)

test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('mnist-data/', train=False, download=True, transform=AlexTransform),
        batch_size=1, shuffle=False)

In [ ]:
# AlexNet
class alexnet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=96, kernel_size=11, stride=4, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(96, 256, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(3, 2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(256, 384, 3, 1, 1),
            nn.ReLU()
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(384, 384, 3, 1, 1),
            nn.ReLU()
        )
        self.conv5 = nn.Sequential(
            nn.Conv2d(384, 256, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(3, 2)
        )

        self.fc1 = nn.Linear(256 * 6 * 6, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 10)

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.conv5(out)
        out = out.view(out.size(0), -1)

        out = F.relu(self.fc1(out))  # 256*6*6 -> 4096
        out = F.dropout(out, 0.5)
        out = F.relu(self.fc2(out))
        out = F.dropout(out, 0.5)
        out = self.fc3(out)


        return out

In [ ]:
cnn = alexnet().to(device)
optimizer = optim.Adam(cnn.parameters())
loss_func = nn.CrossEntropyLoss()

In [ ]:
# Define Hyper parameters
img_size = 28 * 28
hidden_layer_size = 1024
num_classes = 10
net = alexnet().to(device)
# softmax
log_softmax = nn.LogSoftmax(dim=1)

In [ ]:
num_iterations = 15

for j in range(num_iterations):
    for i, (images, labels) in enumerate(train_loader):
        # gives batch data, normalize x when iterate train_loader
        b_x = Variable(images).to(device)   # batch x
        b_y = Variable(labels).to(device)   # batch y
        output = cnn(b_x)
        loss = loss_func(output b_y)
        optimizer.zero_grad() # clear gradients for this training step
        loss.backward() # backpropagation, compute gradients
        optimizer.step() # apply gradients

In [ ]:
def train(cnn, train_loader, epoch, num_epochs):
    
    cnn.train()
        
    # Train the model
    total_step = len(train_loader)
        
    for i, (images, labels) in enumerate(train_loader):
        # gives batch data, normalize x when iterate train_loader
        b_x = Variable(images).to(device)   # batch x
        b_y = Variable(labels).to(device)   # batch y
        output = cnn(b_x)
        loss = loss_func(output b_y)
        optimizer.zero_grad() # clear gradients for this training step
        loss.backward() # backpropagation, compute gradients
        optimizer.step() # apply gradients

        if (i+1) % 10 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch, num_epochs, i + 1, total_step, loss.item()))


In [ ]:
def test(noise=0):

    # Test the model
    cnn.eval()

    with torch.no_grad():

        correct = 0
        total = 0
        i = 0
        soft_out1 = torch.zeros((10000,10))


        for images, labels in test_loader:
            images = images + noise * torch.rand(images.shape)
            test_output = cnn(images.to(device))
            soft_out1[i*batch_size:(i+1)*batch_size,:] = test_output
            pred_y1 = torch.max(test_output, 1)[1].data.squeeze()
            accuracy1 = (pred_y1 == labels.to(device)).sum().item() / float(labels.to(device).size(0))

            i = i + 1

        print('Test Accuracy of the layer1 on the 10000 test images: %.2f' % accuracy1)


    return soft_out1.cpu().numpy()
